# <center>🛩️ **Dashboard Intelligent de Maintenance Prédictive Aéronautique** 🛩️<center>

---

## Contexte du Projet
- **Flotte** : 150 moteurs turbofan
- **Problématique** : Coûts de maintenance imprévus élevés + temps d'arrêt non planifiés
- **Objectif** : Créer un dashboard intelligent pour anticiper les pannes et optimiser les stratégies de maintenance

## Dataset Utilisé
**[NASA Turbofan Engine Degradation Simulation (C-MAPSS)](https://data.nasa.gov/dataset/cmapss-jet-engine-simulated-data)**
- Source : NASA Prognostics Data Repository
- Contenu : Données de capteurs multivariés de moteurs turbofan (température, pression, vibrations...). Le jeu de données simule le comportement run-to-failure des moteurs d’avion, en enregistrant les mesures des capteurs depuis l’état neuf du moteur jusqu’à sa panne.
- Scénario FD001 : ~100 moteurs avec 21 capteurs différents
- Données : Depuis l'état neuf jusqu'à la panne

## Outils et Frameworks
- **Jupyter Notebook** : Environnement d'exécution
- **Plotly Express & Graph Objects** : Visualisations interactives
- **Scikit-learn** : Machine Learning
- **XGBoost** : Gradient Boosting avancé
- **Pandas & NumPy** : Manipulation données
- **Dash** : Framework web pour dashboards

## Plan du projet
1. ✅ Import et Configuration
2. ✅ Chargement et Compréhension des Données
3. ✅ Exploration Avancée (EDA)
4. ✅ Détection d'Anomalies
5. ✅ Feature Engineering
6. ✅ Clustering et Segmentation
7. ✅ Modélisation Prédictive (RUL)
8. ✅ Classification du Risque
9. ✅ KPIs et Évaluation
10. ✅ Dashboard Interactif (4 onglets)
11. ✅ Synthèse Business

---

## **Section 1 : Import des Bibliothèques et Configuration**

In [133]:
# Importer les bibliothèques essentielles
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Data Science & Machine Learning
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import (
    mean_absolute_error, mean_squared_error, r2_score,
    classification_report, confusion_matrix, roc_auc_score, roc_curve,
    precision_score, recall_score, f1_score,
    silhouette_score, davies_bouldin_score
)
from sklearn.feature_selection import mutual_info_regression
from scipy import stats
from sklearn.ensemble import IsolationForest

# Plotly pour visualisations interactives
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

# XGBoost
try:
    import xgboost as xgb
    XGBOOST_AVAILABLE = True
except ImportError:
    XGBOOST_AVAILABLE = False
    print("⚠️ XGBoost non disponible, seuls RandomForest et GradientBoosting seront utilisés")

# Configuration Plotly
pio.templates.default = "plotly_white"

# Paramètres d'affichage pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.4f}'.format)

# Palette de couleurs pour cohérence visuelle
COLOR_PALETTE = {
    'primary': '#3498db',      # Bleu
    'secondary': '#2ecc71',    # Vert
    'warning': '#f39c12',      # Orange
    'danger': '#e74c3c',       # Rouge
    'neutral': '#95a5a6',      # Gris
    'dark': '#2c3e50'          # Bleu foncé
}

---

## **Section 2 : Compréhension et Chargement des Données NASA C-MAPSS**

### Structure du Dataset
- **unit_id** : Identifiant unique du moteur turbofan
- **cycles** : Nombre de cycles opérationnels (variable temporelle)
- **S1 à S21** : Les 21 capteurs mesurent différents paramètres physiques du moteur turbofan :
- **RUL** : Remaining Useful Life (Nombre de cycles restants avant la défaillance du moteur)

| Capteur | Signification                                     | Localisation physique                                                                |
| ------- | ------------------------------------------------- | ------------------------------------------------------------------------------------ |
| **S2**  | **Température de sortie de la turbine BP (LPT)**  | Sortie de la turbine basse pression – mesure de la température des gaz d’échappement |
| **S3**  | **Température de sortie du compresseur HP (HPC)** | Sortie du compresseur haute pression – zone chaude du cœur                           |
| **S4**  | **Température de sortie de la turbine BP (LPT)**  | Turbine basse pression – mesure alternative                                          |
| **S7**  | **Pression de sortie du compresseur HP (HPC)**    | Sortie du compresseur HP – pression de compression                                   |
| **S8**  | **Pression de sortie de la turbine BP (LPT)**     | Sortie de la turbine BP – contre-pression                                            |
| **S9**  | **Pression statique de sortie HPC**               | Mesure de pression du cœur (indépendante)                                            |
| **S11** | **Vitesse corrigée du fan**                       | Vitesse de rotation du fan (corrigée des conditions)                                 |
| **S12** | **Vitesse corrigée du cœur/compresseur**          | Vitesse de rotation du cœur (corrigée des conditions)                                |
| **S13** | **Taux de dilution (Bypass Ratio)**               | Fraction d’air contournant le cœur vs traversant le cœur                             |
| **S14** | **Débit carburant du brûleur**                    | Carburant consommé dans la chambre de combustion                                     |
| **S15** | **Enthalpie de prélèvement (bleed)**              | Énergie extraite de l’air de prélèvement du compresseur                              |
| **S17** | **Vitesse physique du cœur**                      | Vitesse réelle de l’arbre du cœur                                                    |
| **S20** | **Prélèvement de refroidissement HPT**            | Air de refroidissement de la turbine HP                                              |
| **S21** | **Prélèvement de refroidissement LPT**            | Air de refroidissement de la turbine BP                                              |


In [134]:
# Définir le chemin du dataset
DATASET_PATH = Path('dataset')

In [135]:
# Définir les noms de colonnes (NASA C-MAPSS standard)
# Format: unit_id, cycles, 3 operational settings, 21 sensors
operational_settings = ['setting1', 'setting2', 'setting3']
sensor_columns = [f'S{i+1}' for i in range(21)]  # S1 à S21
column_names = ['unit_id', 'cycles'] + operational_settings + sensor_columns

In [136]:
# Charger les données d'entraînement (FD001)
scenario = 'FD001'
train_file = DATASET_PATH / f'train_{scenario}.txt'
data_train = pd.read_csv(train_file, sep=r'\s+', header=None, names=column_names)

# Charger les données de test
test_file = DATASET_PATH / f'test_{scenario}.txt'
data_test = pd.read_csv(test_file, sep=r'\s+', header=None, names=column_names)

# Afficher un aperçu des données
print(data_train.head())
print(data_test.head())


   unit_id  cycles  setting1  setting2  setting3       S1       S2        S3  \
0        1       1   -0.0007   -0.0004  100.0000 518.6700 641.8200 1589.7000   
1        1       2    0.0019   -0.0003  100.0000 518.6700 642.1500 1591.8200   
2        1       3   -0.0043    0.0003  100.0000 518.6700 642.3500 1587.9900   
3        1       4    0.0007    0.0000  100.0000 518.6700 642.3500 1582.7900   
4        1       5   -0.0019   -0.0002  100.0000 518.6700 642.3700 1582.8500   

         S4      S5      S6       S7        S8        S9    S10     S11  \
0 1400.6000 14.6200 21.6100 554.3600 2388.0600 9046.1900 1.3000 47.4700   
1 1403.1400 14.6200 21.6100 553.7500 2388.0400 9044.0700 1.3000 47.4900   
2 1404.2000 14.6200 21.6100 554.2600 2388.0800 9052.9400 1.3000 47.2700   
3 1401.8700 14.6200 21.6100 554.4500 2388.1100 9049.4800 1.3000 47.1300   
4 1406.2200 14.6200 21.6100 554.0000 2388.0600 9055.1500 1.3000 47.2800   

       S12       S13       S14    S15    S16  S17   S18      S19    

### Nettoyer les colonnes inutiles 

In [137]:
# Nettoyer les colonnes inutiles - garder seulement ce qui est nécessaire

# 1. Supprimer toutes les colonnes operational settings (constantes dans FD001)
settings_cols = [col for col in data_train.columns if col.startswith('setting')]
if settings_cols:
    print(f"❌ Paramètres opérationnels supprimés (constants): {settings_cols}")
    data_train = data_train.drop(columns=settings_cols)
    data_test = data_test.drop(columns=settings_cols)

# 2. Identifier les capteurs avec variance nulle ou NaN excessifs
sensor_cols_to_check = [col for col in data_train.columns if col.startswith('S')]
useless_sensors = []

for col in sensor_cols_to_check:
    # Supprimer si constant (variance quasi-nulle, < 0.01)
    if data_train[col].std() < 0.01:
        useless_sensors.append(f"{col} (constant, std={data_train[col].std():.6f})")
    # Supprimer si trop de NaN (>50%)
    elif data_train[col].isna().sum() / len(data_train) > 0.5:
        useless_sensors.append(f"{col} (NaN)")

if useless_sensors:
    cols_to_drop = [s.split(' ')[0] for s in useless_sensors]
    print(f"❌ Capteurs inutiles supprimés: {useless_sensors}")
    data_train = data_train.drop(columns=cols_to_drop)
    data_test = data_test.drop(columns=[col for col in cols_to_drop if col in data_test.columns], errors='ignore')

# 3. Mettre à jour la liste des capteurs actifs
sensor_columns = [col for col in data_train.columns if col.startswith('S')]
print(f"\n✅ Capteurs conservés: {len(sensor_columns)}/21")
print(f"📊 Capteurs actifs: {sensor_columns}")
print(f"📋 Colonnes finales: {list(data_train.columns)}")

❌ Paramètres opérationnels supprimés (constants): ['setting1', 'setting2', 'setting3']
❌ Capteurs inutiles supprimés: ['S1 (constant, std=0.000000)', 'S5 (constant, std=0.000000)', 'S6 (constant, std=0.001389)', 'S10 (constant, std=0.000000)', 'S16 (constant, std=0.000000)', 'S18 (constant, std=0.000000)', 'S19 (constant, std=0.000000)']

✅ Capteurs conservés: 14/21
📊 Capteurs actifs: ['S2', 'S3', 'S4', 'S7', 'S8', 'S9', 'S11', 'S12', 'S13', 'S14', 'S15', 'S17', 'S20', 'S21']
📋 Colonnes finales: ['unit_id', 'cycles', 'S2', 'S3', 'S4', 'S7', 'S8', 'S9', 'S11', 'S12', 'S13', 'S14', 'S15', 'S17', 'S20', 'S21']


### Critères de Suppression

1. **Paramètres Opérationnels Constants (setting1, setting2, setting3)**
   - Dans le scénario FD001, ces paramètres restent constants tout au long du cycle de vie
   - Variance ≈ 0 → aucune information discriminante

2. **Capteurs à Variance Nulle ou Quasi-Nulle (std < 0.01)**
   - **Capteurs supprimés** : S1, S5, S6, S10, S16, S18, S19 (7 capteurs)
   - Ces capteurs n'enregistrent aucune variation significative
   - Ne peuvent pas contribuer à la prédiction de dégradation
   - Exemple : S1 (std=0.000000), S6 (std=0.001389)

3. **Capteurs avec Données Manquantes Excessives (>50%)**
   - Vérification des valeurs NaN
   - Aucun capteur ne dépasse ce seuil dans FD001

### Résultat Final

- **Capteurs conservés** : 14/21 (67%) - S2, S3, S4, S7, S8, S9, S11, S12, S13, S14, S15, S17, S20, S21
- **Avantages** :
  - Réduction de dimensionnalité (moins de bruit)
  - Amélioration des performances des modèles
  - Temps de calcul optimisé
  - Interprétabilité accrue

In [138]:
# Charger les RUL (Remaining Useful Life) cibles
rul_file = DATASET_PATH / f'RUL_{scenario}.txt'
data_rul = pd.read_csv(rul_file, header=None, names=['RUL'])

print(f"✅ Données chargées")
print(f"   Train: {len(data_train):,} observations")
print(f"   Test:  {len(data_test):,} observations")
print(f"   Moteurs uniques: {data_train['unit_id'].nunique()} en train, {data_test['unit_id'].nunique()} en test")

✅ Données chargées
   Train: 20,631 observations
   Test:  13,096 observations
   Moteurs uniques: 100 en train, 100 en test


In [139]:
# Affichage d'un aperçu
print("📊 APERÇU DES DONNÉES - Premières lignes (Moteur #1)")
data_train.head(10)

📊 APERÇU DES DONNÉES - Premières lignes (Moteur #1)


,unit_id,cycles,S2,S3,S4,S7,S8,S9,S11,S12,S13,S14,S15,S17,S20,S21
0,1,1,641.8200,1589.7000,1400.6000,554.3600,2388.0600,9046.1900,47.4700,521.6600,2388.0200,8138.6200,8.4195,392,39.0600,23.4190
1,1,2,642.1500,1591.8200,1403.1400,553.7500,2388.0400,9044.0700,47.4900,522.2800,2388.0700,8131.4900,8.4318,392,39.0000,23.4236
2,1,3,642.3500,1587.9900,1404.2000,554.2600,2388.0800,9052.9400,47.2700,522.4200,2388.0300,8133.2300,8.4178,390,38.9500,23.3442
3,1,4,642.3500,1582.7900,1401.8700,554.4500,2388.1100,9049.4800,47.1300,522.8600,2388.0800,8133.8300,8.3682,392,38.8800,23.3739
4,1,5,642.3700,1582.8500,1406.2200,554.0000,2388.0600,9055.1500,47.2800,522.1900,2388.0400,8133.8000,8.4294,393,38.9000,23.4044
5,1,6,642.1000,1584.4700,1398.3700,554.6700,2388.0200,9049.6800,47.1600,521.6800,2388.0300,8132.8500,8.4108,391,38.9800,23.3669
6,1,7,642.4800,1592.3200,1397.7700,554.3400,2388.0200,9059.1300,47.3600,522.3200,2388.0300,8132.3200,8.3974,392,39.1000,23.3774
7,1,8,642.5600,1582.9600,1400.9700,553.8500,2388.0000,9040.8000,47.2400,522.4700,2388.0300,8131.0700,8.4076,391,38.9700,23.3106
8,1,9,642.1200,1590.9800,1394.8000,553.6900,2388.0500,9046.4600,47.2900,521.7900,2388.0500,8125.6900,8.3728,392,39.0500,23.4066
9,1,10,641.7100,1591.2400,1400.4600,553.5900,2388.0500,9051.7000,47.0300,521.7900,2388.0600,8129.3800,8.4286,393,38.9500,23.4694


In [140]:
# Créer le RUL pour les données d'entraînement
data_train['rul'] = data_train.groupby('unit_id')['cycles'].transform(lambda x: x.max() - x)

print("\nSTATISTIQUES DESCRIPTIVES")
print(data_train.describe().round(3))

print("\nDISTRIBUTION DES CYCLES")
cycles_per_unit = data_train.groupby('unit_id')['cycles'].max()
print(f"   Min cycles: {cycles_per_unit.min()}")
print(f"   Max cycles: {cycles_per_unit.max():.0f}")
print(f"   Moyenne: {cycles_per_unit.mean():.0f}")

print("\nRUL (Remaining Useful Life)")
print(f"   Min RUL: {data_train['rul'].min()}")
print(f"   Max RUL: {data_train['rul'].max():.0f}")
print(f"   Moyenne: {data_train['rul'].mean():.0f}")


STATISTIQUES DESCRIPTIVES
         unit_id     cycles         S2         S3         S4         S7  \
count 20631.0000 20631.0000 20631.0000 20631.0000 20631.0000 20631.0000   
mean     51.5070   108.8080   642.6810  1590.5230  1408.9340   553.3680   
std      29.2280    68.8810     0.5000     6.1310     9.0010     0.8850   
min       1.0000     1.0000   641.2100  1571.0400  1382.2500   549.8500   
25%      26.0000    52.0000   642.3250  1586.2600  1402.3600   552.8100   
50%      52.0000   104.0000   642.6400  1590.1000  1408.0400   553.4400   
75%      77.0000   156.0000   643.0000  1594.3800  1414.5550   554.0100   
max     100.0000   362.0000   644.5300  1616.9100  1441.4900   556.0600   

              S8         S9        S11        S12        S13        S14  \
count 20631.0000 20631.0000 20631.0000 20631.0000 20631.0000 20631.0000   
mean   2388.0970  9065.2430    47.5410   521.4130  2388.0960  8143.7530   
std       0.0710    22.0830     0.2670     0.7380     0.0720    19.0760 

---

## **Section 3 : Exploration Avancée (EDA) et Analyse Temporelle**

### 3.1 Distribution de la Progression des Cycles (Boxplot et Histogramme)

In [141]:
fig_hist = px.histogram(
    data_train,
    x='cycles',
    nbins=30,
    title='Distribution de la Progression des Cycles (FD001)',
    labels={'cycles': 'Nombre de cycles'},
    color_discrete_sequence=[COLOR_PALETTE['primary']],
    marginal='box' # Ajoute un box plot au-dessus pour le contexte
)

fig_hist.update_layout(
    template='plotly_white',
    title_x=0.5,
    xaxis_title='Cycle de fonctionnement',
    yaxis_title='Fréquence (Nombre d\'enregistrements)',
    bargap=0.1,
    height=500
)

fig_hist.show()

### 1. Le Box Plot (Statistiques Clés)

Ce graphique montre **quand** les moteurs tombent en panne.

* **Médiane (~105-110 cycles) :** La moitié des moteurs s'arrêtent avant ce seuil. C'est la durée de vie "typique".
* **Variabilité (Boîte) :** 50% des pannes surviennent entre 50 et 155 cycles. L'écart est large, ce qui rend la maintenance préventive fixe risquée.
* **Valeurs Aberrantes (Points isolés) :** Certains moteurs sont exceptionnels et atteignent **362 cycles**.

### 2. L'Histogramme (Volume de Données)

Ce graphique montre **combien** de données vous possédez pour chaque stade de vie.

**Pente Descendante :** La fréquence chute à mesure que les cycles augmentent car les moteurs "meurent".

=> Le dataset est **asymétrique à droite**.

### 3.2 Matrice de corrélation entre capteurs (heatmap)

In [142]:
# 3.2 Heatmap avec labels de corrélation
correlation_matrix = data_train[sensor_columns].corr()

fig_heatmap_labeled = go.Figure(data=go.Heatmap(
    z=correlation_matrix.values,
    x=sensor_columns,
    y=sensor_columns,
    colorscale='RdBu_r',
    zmin=-1, zmax=1,
    colorbar=dict(title="Corrélation"),
    text=correlation_matrix.round(2).values,
    texttemplate="%{text}",
    textfont={"size":10}
))

fig_heatmap_labeled.update_layout(
    title='Matrice de Corrélation entre Capteurs',
    title_font_size=16,
    title_x=0.5,
    width=900,
    height=800,
    template='plotly_white',
    xaxis=dict(title_font_size=12, tickfont_size=10),
    yaxis=dict(title_font_size=12, tickfont_size=10),
    margin=dict(l=100, r=100, t=100, b=100)
)
fig_heatmap_labeled.show()

#### 1. Anti-corrélations Significatives & Synchronisations Fortes

Ces paires capturent la **dynamique du moteur**:

| Paire | Corrélation | Interprétation |
|-------|-------------|----------------|
| **S4 ↔ S7** | **-0.7931** | Pression vs vitesse: compensation inverse |
| **S12 ↔ S7** | **+0.8127** | Carburant vs vitesse: **synchronisation directe (couplage thermodynamique)** |
| **S11 ↔ S12** | **-0.8469** | Débit vs carburant: **plus fort signal anti-corrélation** |
| **S12 ↔ S13** | **-0.79** | Couple vs débit: déséquilibre opérationnel |
| **S3 ↔ S7** | **-0.66** | Température vs vitesse: relation physique |

---

#### 2. Peu/Pas de Corrélation (Blancs/Pales)

- **S9 avec presque tous** (r ≈ 0.3 partout): **Isolation quasi-complète** → capteur complètement indépendant
  
- **S14 avec la plupart** (r ≈ 0.18-0.25): Peu de redondance → complément orthogonal

**→ Impact positif**: Réduisent multicollinéarité, ajoutent diversité sans bruit

### 3.3 Analyse Temporelle : S7 (Vitesse Fan) vs S12 (Débit Carburant)

- **Corrélation** : **+0.8127** (fortement positive/synchrone)
- **Signification physique** : Synchronisation moteur — S7 et S12 augmentent/diminuent ENSEMBLE → couplage thermodynamique direct
- **Utilité RUL** : Détection de défaillances (perte de synchronisation = premier symptôme de dégradation)

In [163]:
# 3.3 Analyse temporelle : S7 (Fan Speed) vs S12 (Fuel Flow) - Paire clé de dégradation
# Sélectionner les 2 premiers moteurs pour visualisation claire
sample_units_s7_s12 = data_train['unit_id'].unique()[:2]

fig_s7_s12 = make_subplots(
    rows=2, cols=1,
    subplot_titles=[f'Moteur {uid}' for uid in sample_units_s7_s12],
    specs=[[{'secondary_y': True}],
           [{'secondary_y': True}]]
)

row_col_pairs = [(1,1), (2,1)]

for idx, (unit_id, (row, col)) in enumerate(zip(sample_units_s7_s12, row_col_pairs)):
    df_motor = data_train[data_train['unit_id'] == unit_id].sort_values('cycles')
    
    # Normaliser pour voir la corrélation (plot sur même échelle)
    s7_norm = (df_motor['S7'] - df_motor['S7'].min()) / (df_motor['S7'].max() - df_motor['S7'].min())
    s12_norm = (df_motor['S12'] - df_motor['S12'].min()) / (df_motor['S12'].max() - df_motor['S12'].min())
    
    # S7 (Fan Speed) - axe principal
    fig_s7_s12.add_trace(
        go.Scatter(
            x=df_motor['cycles'],
            y=s7_norm,
            mode='lines',
            name=f'S7 (Fan Speed)',
            line=dict(color=COLOR_PALETTE['primary'], width=2.5),
            hovertemplate='<b>S7 - Fan Speed</b><br>Cycles: %{x}<br>Normalized: %{y:.3f}<extra></extra>'
        ),
        row=row, col=col, secondary_y=False
    )
    
    # S12 (Fuel Flow) - axe secondaire (direct, car corrélation positive +0.8127)
    fig_s7_s12.add_trace(
        go.Scatter(
            x=df_motor['cycles'],
            y=s12_norm,  # Affichage direct (corrélation positive, synchronisation)
            mode='lines',
            name=f'S12 (Fuel Flow)',
            line=dict(color=COLOR_PALETTE['danger'], width=2.5, dash='dash'),
            hovertemplate='<b>S12 - Fuel Flow</b><br>Cycles: %{x}<br>Normalized: %{y:.3f}<extra></extra>'
        ),
        row=row, col=col, secondary_y=False
    )
    
    # RUL en arrière-plan (zone grisée)
    rul_max = df_motor['rul'].max()
    fig_s7_s12.add_vrect(
        x0=df_motor[df_motor['rul'] <= RUL_THRESHOLD_WARNING]['cycles'].min() if len(df_motor[df_motor['rul'] <= RUL_THRESHOLD_WARNING]) > 0 else df_motor['cycles'].max(),
        x1=df_motor['cycles'].max(),
        fillcolor=COLOR_PALETTE['danger'],
        opacity=0.1,
        line_width=0,
        layer="below",
        row=row, col=col
    )

# Mise en page
fig_s7_s12.update_xaxes(title_text='Cycles', title_font_size=11, tickfont_size=10, showgrid=True, gridcolor='lightgray')
fig_s7_s12.update_yaxes(title_text='Normalized Value', title_font_size=11, tickfont_size=10, showgrid=True, gridcolor='lightgray')

fig_s7_s12.update_layout(
    title_text='Analyse Temporelle : S7 (Vitesse Fan) vs S12 (Débit Carburant)<br><sub>Synchronisation directe (r=+0.8127) | Zone rouge = RUL faible</sub>',
    title_font_size=16,
    title_x=0.5,
    height=800,
    width=1200,
    template='plotly_white',
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.02,
        xanchor="center",
        x=0.5,
        font_size=10,
        bgcolor='rgba(255,255,255,0.8)'
    ),
    margin=dict(l=70, r=40, t=120, b=80)
)

fig_s7_s12.show()

---

## **Section 4 : Détection d'Anomalies et Signaux d'Alerte**

In [146]:
df_eda = data_train.copy()

In [147]:
# 4.1 Détection Z-score
print("Détection Z-score (seuil = 3)")
z_scores = np.abs(stats.zscore(data_train[sensor_columns], nan_policy='omit'))
anomalies_zscore = (z_scores > 3).sum(axis=1) > 0
df_eda['is_anomaly_zscore'] = anomalies_zscore
print(f"Anomalies détectées: {anomalies_zscore.sum()} ({100*anomalies_zscore.sum()/len(data_train):.2f}%)")

Détection Z-score (seuil = 3)
Anomalies détectées: 891 (4.32%)


In [148]:
# 4.2 Isolation Forest
print("Détection Isolation Forest")
iso_forest = IsolationForest(contamination=0.05, random_state=42)
anomalies_if = iso_forest.fit_predict(data_train[sensor_columns])
df_eda['is_anomaly_if'] = anomalies_if == -1
print(f"Anomalies détectées: {(anomalies_if == -1).sum()}")

Détection Isolation Forest
Anomalies détectées: 1032


In [149]:
# 4.3 Score d'anomalie composite
df_eda['anomaly_score'] = (anomalies_zscore.astype(int) + (anomalies_if == -1).astype(int)) / 2
print(f"Score d'Anomalie Composite")
print(f"Score moyen: {df_eda['anomaly_score'].mean():.4f}")
print(f"Moteurs avec au moins 1 anomalie: {(df_eda['anomaly_score'] > 0).sum()}")

Score d'Anomalie Composite
Score moyen: 0.0466
Moteurs avec au moins 1 anomalie: 1291


- **Z-score (seuil 3σ)** : détecte les pics isolés capteur par capteur. Résultat : 4,3% des cycles anormaux.
- **Isolation Forest** : repère les combinaisons inhabituelles multi-capteurs. Résultat : 1 032 cycles.
- **Score composite** : ne retient que ce qui est confirmé (consensus). Résultat : 0,047 de score moyen, 1 291 moteurs touchés.
- **Règle d’usage** : déclencher maintenance sur les anomalies validées par ≥2 méthodes; surveiller les alertes uniques.
- **Bénéfice** : moins de faux positifs, meilleure détection des dégradations réelles du turbofan.

### Visualisation des anomalies

In [164]:
# 4.4 Visualisation des anomalies
# Utiliser des capteurs avec plus de variance pour mieux visualiser
top_variance_sensors = data_train[sensor_columns].std().nlargest(2).index.tolist()
sensor_x, sensor_y = top_variance_sensors[0], top_variance_sensors[1]

# Créer un échantillon avec surreprésentation des anomalies
df_normal = df_eda[df_eda['anomaly_score'] == 0].sample(min(3000, len(df_eda[df_eda['anomaly_score'] == 0])))
df_anomalies = df_eda[df_eda['anomaly_score'] > 0]
df_viz = pd.concat([df_normal, df_anomalies])

print(f"Visualisation avec {sensor_x} et {sensor_y}")
print(f"Points normaux: {len(df_normal):,}")
print(f"Points anomalies: {len(df_anomalies):,}")

fig_anomaly = px.scatter(
    df_viz,
    x=sensor_x,
    y=sensor_y,
    color='anomaly_score',
    color_continuous_scale='YlOrRd',
    title=f'Détection d\'Anomalies - Espace des Capteurs {sensor_x}-{sensor_y}',
    labels={sensor_x: f'Capteur {sensor_x}', sensor_y: f'Capteur {sensor_y}', 'anomaly_score': 'Score Anomalie'},
    opacity=0.6,
    hover_data=['unit_id', 'cycles', 'rul']
)
fig_anomaly.update_layout(
    height=600,
    template='plotly_white',
    coloraxis_colorbar=dict(title="Score<br>Anomalie")
)
fig_anomaly.show()

Visualisation avec S9 et S14
Points normaux: 3,000
Points anomalies: 1,291


- Les capteurs affichés (S9 et S14) sont choisis automatiquement comme les **2 plus variables** du jeu de données (`std().nlargest(2)`), pour maximiser la séparation visuelle normal/anomalie.
- Points tracés : échantillon équilibré (beaucoup de points normaux + toutes les anomalies) pour bien voir la frontière.
- Couleur = `anomaly_score` : plus c'est foncé/rouge, plus l'observation est considérée anormale.
- Taille des groupes : Normal = échantillon de 3k max ; Anomalies = toutes les anomalies.

---

## **Section 5 : Feature Engineering pour Séries Temporelles**

In [ ]:
# 5.1 Création de features glissantes

def create_rolling_features(data, windows=[5, 10, 20], columns=sensor_columns):
    """Crée des features statistiques glissantes"""
    df_features = data.copy()
    
    for window in windows:
        for col in columns:
            # Moyenne glissante
            df_features[f'{col}_mean_{window}'] = df_features.groupby('unit_id')[col].transform(
                lambda x: x.rolling(window=window, min_periods=1).mean()
            )
            # Écart-type glissant
            df_features[f'{col}_std_{window}'] = df_features.groupby('unit_id')[col].transform(
                lambda x: x.rolling(window=window, min_periods=1).std().fillna(0)
            )
    
    return df_features

print("Création de features glissantes...")
df_features = create_rolling_features(df_eda, windows=[5, 10, 20])
print(f"Nombre initial de colonnes: {len(df_eda.columns)}")
print(f"Nombre après feature engineering: {len(df_features.columns)}")

Création de features glissantes...
Nombre initial de colonnes: 20
Nombre après feature engineering: 104


In [ ]:
# 5.2 Sélection de features via Mutual Information
print("Sélection de features via Mutual Information...")

feature_cols_for_selection = [col for col in df_features.columns 
                               if col not in ['unit_id', 'cycles', 'scenario', 'rul', 
                                              'is_anomaly_zscore', 'is_anomaly_if', 'anomaly_score']]

mi_scores = mutual_info_regression(
    df_features[feature_cols_for_selection].fillna(0),
    df_features['rul'],
    random_state=42
)

mi_df = pd.DataFrame({
    'feature': feature_cols_for_selection,
    'mi_score': mi_scores
}).sort_values('mi_score', ascending=False)

print(f"\nTop 15 features:")
print(mi_df.head(15).to_string(index=False))

top_features = mi_df.head(30)['feature'].tolist()
print(f"\n{len(top_features)} features sélectionnées")

Sélection de features via Mutual Information...

Top 15 features:
    feature  mi_score
S11_mean_20    0.6962
S11_mean_10    0.6956
 S4_mean_20    0.6825
 S11_mean_5    0.6804
 S4_mean_10    0.6682
S17_mean_20    0.6638
 S2_mean_20    0.6497
S15_mean_10    0.6479
S21_mean_20    0.6471
S15_mean_20    0.6466
  S4_mean_5    0.6461
S21_mean_10    0.6446
 S2_mean_10    0.6380
S20_mean_10    0.6323
 S7_mean_20    0.6279

30 features sélectionnées


In [ ]:
# 5.3 Normalisation des données
print("Normalisation avec StandardScaler...")
scaler = StandardScaler()
df_scaled = df_features.copy()
df_scaled[top_features] = scaler.fit_transform(df_features[top_features].fillna(0))

# 5.4 Préparation pour modélisation
X_train = df_scaled[top_features].values
y_train = df_scaled['rul'].values

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"Samples: {len(X_train):,}")

Normalisation avec StandardScaler...
X_train shape: (20631, 30)
y_train shape: (20631,)
Samples: 20,631


---

## **Section 6 : Clustering et Segmentation de la Flotte**

In [ ]:
# 6.1 PCA pour réduction de dimensionnalité
print("Réduction de dimensionnalité avec PCA...")
pca = PCA(n_components=10, random_state=42)
X_pca = pca.fit_transform(X_train)
print(f"Variance expliquée: {pca.explained_variance_ratio_.cumsum()[-1]:.2%}")

Réduction de dimensionnalité avec PCA...
Variance expliquée: 98.85%


In [ ]:
# 6.2 Détermination du nombre optimal de clusters
print("Détermination du nombre optimal de clusters...")
inertias = []
silhouette_scores = []
K_range = range(2, 11)

for k in K_range:
    kmeans_temp = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels_temp = kmeans_temp.fit_predict(X_pca)
    inertias.append(kmeans_temp.inertia_)
    silhouette_scores.append(silhouette_score(X_pca, labels_temp))

# Visualiser l'Elbow
fig_elbow = make_subplots(rows=1, cols=2, subplot_titles=('Inertie (Elbow)', 'Silhouette Score'))

fig_elbow.add_trace(
    go.Scatter(x=list(K_range), y=inertias, mode='lines+markers', name='Inertie',
               line=dict(color=COLOR_PALETTE['primary'], width=3)),
    row=1, col=1
)

fig_elbow.add_trace(
    go.Scatter(x=list(K_range), y=silhouette_scores, mode='lines+markers', name='Silhouette',
               line=dict(color=COLOR_PALETTE['secondary'], width=3)),
    row=1, col=2
)

fig_elbow.update_xaxes(title_text='Nombre de clusters (k)', title_font_size=13, row=1, col=1)
fig_elbow.update_xaxes(title_text='Nombre de clusters (k)', title_font_size=13, row=1, col=2)
fig_elbow.update_yaxes(title_text='Inertie', title_font_size=13, row=1, col=1)
fig_elbow.update_yaxes(title_text='Silhouette Score', title_font_size=13, row=1, col=2)
fig_elbow.update_layout(
    height=500,
    width=1100,
    title_text='Sélection du nombre optimal de clusters',
    title_font_size=16,
    title_x=0.5,
    template='plotly_white',
    showlegend=True,
    margin=dict(l=60, r=40, t=80, b=60)
)
fig_elbow.show()

# Sélectionner k optimal
optimal_k = list(K_range)[np.argmax(silhouette_scores)]
print(f"Nombre optimal de clusters: {optimal_k}")

Détermination du nombre optimal de clusters...


Nombre optimal de clusters: 2


In [ ]:
# 6.3 K-Means avec k optimal
print(f"Clustering avec K-Means (k={optimal_k})...")
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
df_features['cluster'] = kmeans.fit_predict(X_pca)

print(f"Distribution des clusters:")
print(df_features['cluster'].value_counts().sort_index())

Clustering avec K-Means (k=2)...
Distribution des clusters:
cluster
0     7314
1    13317
Name: count, dtype: int64


In [165]:
# 6.4 Visualisation PCA clusters
fig_clusters_pca = px.scatter(
    df_features,
    x=X_pca[:, 0],
    y=X_pca[:, 1],
    color='cluster',
    title='Clustering K-Means (Projection PCA)',
    labels={'x': 'Composante Principale 1', 'y': 'Composante Principale 2', 'cluster': 'Cluster'},
    color_discrete_sequence=list(COLOR_PALETTE.values())
)
fig_clusters_pca.update_layout(
    height=600,
    width=900,
    template='plotly_white',
    title_font_size=16,
    title_x=0.5,
    xaxis=dict(showgrid=True, gridcolor='lightgray', title_font_size=14),
    yaxis=dict(showgrid=True, gridcolor='lightgray', title_font_size=14),
    legend=dict(title_font_size=13, font_size=11),
    margin=dict(l=60, r=40, t=80, b=60)
)
fig_clusters_pca.show()

1. Séparation des clusters sur la PC1

    La caractéristique la plus marquante est que la séparation entre les clusters **jaune** et **bleu foncé** se fait presque verticalement le long de l’axe de la **Composante Principale 1 (PC1)**.

* **PC1** représente généralement la direction de **variance maximale** dans les données.
* Le fait que la séparation se produise approximativement à une valeur donnée de PC1 suggère que les variables contribuant le plus à PC1 sont les **principaux facteurs** qui pilotent la décision de clustering.

2. Chevauchement et densité

* On observe une **forte densité de points** à l’endroit où les deux clusters se rejoignent.
* Cela indique que les données sont **relativement continues**, plutôt que structurées autour d’un « vide » ou d’un espace clairement séparateur entre les groupes.

3. Valeurs aberrantes (outliers)

* Plusieurs **valeurs aberrantes** sont visibles, en particulier dans le cluster jaune (par exemple, des points situés aux extrémités du nuage).
* Comme **K-means** utilise la **moyenne (centroïde)** pour définir les clusters, il peut être sensible à ces outliers.
* Toutefois, étant donné la **forte densité** des nuages principaux, ces valeurs extrêmes n’ont probablement **pas déplacé significativement les centroïdes**.


In [168]:
# 6.5 Distribution du RUL par cluster
fig_rul_cluster = px.box(
    df_features,
    x='cluster',
    y='rul',
    color='cluster',
    title='Distribution du RUL par Cluster',
    labels={'rul': 'Remaining Useful Life (cycles)', 'cluster': 'Cluster'},
    points='outliers',
    color_discrete_sequence=list(COLOR_PALETTE.values())
)
fig_rul_cluster.update_layout(
    height=550,
    width=900,
    template='plotly_white',
    title_font_size=16,
    title_x=0.5,
    xaxis=dict(title_font_size=14, tickfont_size=12),
    yaxis=dict(title_font_size=14, tickfont_size=12, showgrid=True, gridcolor='lightgray'),
    showlegend=False,
    margin=dict(l=80, r=40, t=80, b=60)
)
fig_rul_cluster.show()

Ce graphique illustre la distribution de la **Remaining Useful Life (RUL)** pour les deux clusters identifiés lors de l’analyse **K-Means** précédente.

1. Séparation significative des états de santé

 Les deux clusters présentent une distinction très nette en termes de valeurs de RUL :

* **Cluster 0 (Vert)** : représente des composants/moteurs dans un **état dégradé**.
  La médiane de la RUL est relativement faible (environ **40 cycles**), et la majorité des données (la « boîte » du boxplot) se situe en dessous de **75 cycles**.

* **Cluster 1 (Bleu)** : représente des composants dans un **état sain ou en début de vie**.
  La médiane de la RUL est beaucoup plus élevée (environ **140 cycles**), avec une distribution qui s’étend jusqu’à **plus de 350 cycles**.


2. Variance et distribution

* **Le cluster 1** présente un **intervalle interquartile (IQR)** nettement plus large (hauteur de la boîte bleue) que le cluster 0.
  Cela est attendu, car des machines « saines » peuvent avoir une durée de vie restante très variable selon leur état initial et l’intensité de leur utilisation.

* **Le cluster 0** est beaucoup plus **concentré vers le bas de l’échelle**.
  Cela suggère qu’à mesure que les machines approchent de la défaillance, leurs profils de capteurs (qui ont guidé le clustering) deviennent **plus similaires entre eux** et **clairement distincts** de ceux des machines en bon état.


---

## **Section 7 : Modélisation Prédictive du RUL**

In [ ]:
# 7.1 Split train/test
print("Préparation des données...")
X_train_model, X_test_model, y_train_model, y_test_model = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)
print(f"Train: {X_train_model.shape[0]:,} samples")
print(f"Test: {X_test_model.shape[0]:,} samples")

Préparation des données...
Train: 16,504 samples
Test: 4,127 samples


In [ ]:
# 7.2 Entraînement des modèles
print("Entraînement des modèles...\n")

models = {}
predictions = {}
performance = {}

# Random Forest
print("Random Forest Regressor...")
rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=20,
    min_samples_split=5,
    random_state=42,
    n_jobs=-1
)
rf_model.fit(X_train_model, y_train_model)
y_pred_rf = rf_model.predict(X_test_model)

mae_rf = mean_absolute_error(y_test_model, y_pred_rf)
rmse_rf = np.sqrt(mean_squared_error(y_test_model, y_pred_rf))
r2_rf = r2_score(y_test_model, y_pred_rf)

models['RandomForest'] = rf_model
predictions['RandomForest'] = y_pred_rf
performance['RandomForest'] = {'MAE': mae_rf, 'RMSE': rmse_rf, 'R2': r2_rf}
print(f"  MAE: {mae_rf:.2f}, RMSE: {rmse_rf:.2f}, R²: {r2_rf:.4f}")

# Gradient Boosting
print("Gradient Boosting Regressor...")
gb_model = GradientBoostingRegressor(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    random_state=42
)
gb_model.fit(X_train_model, y_train_model)
y_pred_gb = gb_model.predict(X_test_model)

mae_gb = mean_absolute_error(y_test_model, y_pred_gb)
rmse_gb = np.sqrt(mean_squared_error(y_test_model, y_pred_gb))
r2_gb = r2_score(y_test_model, y_pred_gb)

models['GradientBoosting'] = gb_model
predictions['GradientBoosting'] = y_pred_gb
performance['GradientBoosting'] = {'MAE': mae_gb, 'RMSE': rmse_gb, 'R2': r2_gb}
print(f"  MAE: {mae_gb:.2f}, RMSE: {rmse_gb:.2f}, R²: {r2_gb:.4f}")

# XGBoost
if XGBOOST_AVAILABLE:
    print("XGBoost Regressor...")
    xgb_model = xgb.XGBRegressor(
        n_estimators=100,
        max_depth=5,
        learning_rate=0.1,
        random_state=42
    )
    xgb_model.fit(X_train_model, y_train_model)
    y_pred_xgb = xgb_model.predict(X_test_model)
    
    mae_xgb = mean_absolute_error(y_test_model, y_pred_xgb)
    rmse_xgb = np.sqrt(mean_squared_error(y_test_model, y_pred_xgb))
    r2_xgb = r2_score(y_test_model, y_pred_xgb)
    
    models['XGBoost'] = xgb_model
    predictions['XGBoost'] = y_pred_xgb
    performance['XGBoost'] = {'MAE': mae_xgb, 'RMSE': rmse_xgb, 'R2': r2_xgb}
    print(f"  MAE: {mae_xgb:.2f}, RMSE: {rmse_xgb:.2f}, R²: {r2_xgb:.4f}")

Entraînement des modèles...

Random Forest Regressor...
  MAE: 19.99, RMSE: 28.44, R²: 0.8229
Gradient Boosting Regressor...
  MAE: 24.83, RMSE: 33.78, R²: 0.7502
XGBoost Regressor...
  MAE: 24.64, RMSE: 33.29, R²: 0.7574


In [ ]:
# 7.3 Comparaison des modèles
print("COMPARAISON DES MODÈLES:")
perf_df = pd.DataFrame(performance).T
print(perf_df)

COMPARAISON DES MODÈLES:
                     MAE    RMSE     R2
RandomForest     19.9857 28.4426 0.8229
GradientBoosting 24.8311 33.7803 0.7502
XGBoost          24.6353 33.2929 0.7574


In [166]:
# 7.4 Visualisation des performances
fig_models = px.bar(
    perf_df.reset_index().melt(id_vars='index', var_name='Métrique', value_name='Score'),
    x='index',
    y='Score',
    color='Métrique',
    title='Comparaison des Performances des Modèles',
    labels={'index': 'Modèle', 'Score': 'Score'},
    barmode='group',
    color_discrete_map={'MAE': COLOR_PALETTE['warning'], 'RMSE': COLOR_PALETTE['danger'], 'R2': COLOR_PALETTE['secondary']}
)
fig_models.update_layout(
    height=550,
    width=900,
    template='plotly_white',
    title_font_size=16,
    title_x=0.5,
    xaxis=dict(title_font_size=14, tickfont_size=12),
    yaxis=dict(title_font_size=14, tickfont_size=12, showgrid=True, gridcolor='lightgray'),
    legend=dict(title_font_size=13, font_size=11, orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    margin=dict(l=60, r=40, t=100, b=60)
)
fig_models.show()

In [ ]:
# Sélectionner le meilleur modèle
best_model_name = perf_df['R2'].idxmax()
best_predictions = predictions[best_model_name]

print(f"Meilleur modèle: {best_model_name}")
print(f"   MAE: {perf_df.loc[best_model_name, 'MAE']:.2f}")
print(f"   R²: {perf_df.loc[best_model_name, 'R2']:.4f}")

Meilleur modèle: RandomForest
   MAE: 19.99
   R²: 0.8229


---

## **Section 8 : Classification du Risque Moteur**

In [ ]:
# 8.1 Définir les seuils de risque
RUL_THRESHOLD_CRITICAL = 10   # Rouge: RUL <= 10
RUL_THRESHOLD_WARNING = 30    # Jaune: 10 < RUL <= 30

print("Définition des classes de risque...")
print(f"  Seuil critique: RUL <= {RUL_THRESHOLD_CRITICAL}")
print(f"  Seuil alerte: RUL <= {RUL_THRESHOLD_WARNING}")

# Classification binaire pour courbe ROC
y_binary_risk = (y_train_model <= RUL_THRESHOLD_WARNING).astype(int)
y_test_binary_risk = (y_test_model <= RUL_THRESHOLD_WARNING).astype(int)

print(f"\nDistribution des classes (Train):")
print(f"  Sains (RUL > {RUL_THRESHOLD_WARNING}): {(y_binary_risk == 0).sum()}")
print(f"  À risque (RUL <= {RUL_THRESHOLD_WARNING}): {(y_binary_risk == 1).sum()}")
print(f"\nDistribution des classes (Test):")
print(f"  Sains (RUL > {RUL_THRESHOLD_WARNING}): {(y_test_binary_risk == 0).sum()}")
print(f"  À risque (RUL <= {RUL_THRESHOLD_WARNING}): {(y_test_binary_risk == 1).sum()}")

Définition des classes de risque...
  Seuil critique: RUL <= 10
  Seuil alerte: RUL <= 30

Distribution des classes (Train):
  Sains (RUL > 30): 13988
  À risque (RUL <= 30): 2516

Distribution des classes (Test):
  Sains (RUL > 30): 3543
  À risque (RUL <= 30): 584


In [ ]:
# 8.2 Entraîner un classifieur
print("Entraînement du classifieur de risque...")
rf_classifier = RandomForestClassifier(
    n_estimators=100,
    max_depth=15,
    random_state=42,
    n_jobs=-1
)
rf_classifier.fit(X_train_model, y_binary_risk)
y_pred_risk = rf_classifier.predict(X_test_model)

# Vérifier que les deux classes sont présentes pour predict_proba
if len(rf_classifier.classes_) < 2:
    print("Une seule classe détectée, ajustement du seuil...")
    # Si une seule classe, utiliser decision_function ou créer probabilités manuelles
    y_pred_proba_risk = rf_classifier.predict_proba(X_test_model)[:, 0]
else:
    # Les deux classes sont présentes
    y_pred_proba_risk = rf_classifier.predict_proba(X_test_model)[:, 1]

# Évaluation
precision = precision_score(y_test_binary_risk, y_pred_risk)
recall = recall_score(y_test_binary_risk, y_pred_risk)
f1 = f1_score(y_test_binary_risk, y_pred_risk)
roc_auc = roc_auc_score(y_test_binary_risk, y_pred_proba_risk)

print(f"Métriques:")
print(f"   Précision: {precision:.4f}")
print(f"   Recall: {recall:.4f}")
print(f"   F1-Score: {f1:.4f}")
print(f"   ROC-AUC: {roc_auc:.4f}")

Entraînement du classifieur de risque...
Métriques:
   Précision: 0.9348
   Recall: 0.8836
   F1-Score: 0.9085
   ROC-AUC: 0.9962


In [ ]:
# 8.3 Matrice de confusion
cm = confusion_matrix(y_test_binary_risk, y_pred_risk)
fig_cm = go.Figure(data=go.Heatmap(
    z=cm,
    x=['Sain', 'À Risque'],
    y=['Sain', 'À Risque'],
    text=cm,
    texttemplate='%{text}',
    textfont={"size": 16},
    colorscale='RdYlGn_r',
    showscale=True,
    colorbar=dict(title="Nombre")
))
fig_cm.update_layout(
    title='Matrice de Confusion - Classification du Risque',
    title_font_size=16,
    title_x=0.5,
    xaxis_title='Prédiction',
    yaxis_title='Réalité',
    xaxis=dict(title_font_size=14, tickfont_size=12, side='bottom'),
    yaxis=dict(title_font_size=14, tickfont_size=12),
    height=550,
    width=600,
    template='plotly_white',
    margin=dict(l=80, r=40, t=80, b=60)
)
fig_cm.show()

In [ ]:
# 8.4 Courbe ROC
fpr, tpr, thresholds = roc_curve(y_test_binary_risk, y_pred_proba_risk)

fig_roc = px.line(
    x=fpr,
    y=tpr,
    title='Courbe ROC - Classification du Risque',
    labels={'x': 'Taux de Faux Positifs', 'y': 'Taux de Vrais Positifs'}
)
fig_roc.update_traces(line=dict(color=COLOR_PALETTE['primary'], width=3))
fig_roc.add_trace(
    go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Aléatoire (AUC=0.5)',
               line=dict(dash='dash', color='gray', width=2))
)
fig_roc.add_annotation(
    x=0.6, y=0.3,
    text=f'AUC = {roc_auc:.3f}',
    showarrow=False,
    font=dict(size=14, color=COLOR_PALETTE['dark']),
    bgcolor="white",
    bordercolor=COLOR_PALETTE['primary'],
    borderwidth=2
)
fig_roc.update_layout(
    height=600,
    width=700,
    template='plotly_white',
    title_font_size=16,
    title_x=0.5,
    xaxis=dict(title_font_size=14, tickfont_size=12, showgrid=True, gridcolor='lightgray', range=[-0.05, 1.05]),
    yaxis=dict(title_font_size=14, tickfont_size=12, showgrid=True, gridcolor='lightgray', range=[-0.05, 1.05]),
    legend=dict(x=0.7, y=0.1, font_size=11),
    margin=dict(l=60, r=40, t=80, b=60)
)
fig_roc.show()

---

## **Section 9 : KPIs Opérationnels et Financiers**

### 9.1 Prédictions pour tous les moteurs

In [ ]:
# 9.1 Prédictions pour tous les moteurs
y_pred_all = models[best_model_name].predict(X_train)
df_features['rul_predicted'] = y_pred_all
df_features['rul_error'] = np.abs(df_features['rul'] - df_features['rul_predicted'])

# KPIs Opérationnels
print("KPIs OPÉRATIONNELS:\n")

moteurs_a_risque = (df_features['rul_predicted'] <= RUL_THRESHOLD_WARNING).sum()
pct_risque = 100 * moteurs_a_risque / len(df_features)
moteurs_sains = (df_features['rul_predicted'] > RUL_THRESHOLD_WARNING).sum()
moteurs_critiques = (df_features['rul_predicted'] <= RUL_THRESHOLD_CRITICAL).sum()

print(f"Moteurs à Risque (RUL ≤ {RUL_THRESHOLD_WARNING}):")
print(f"   Sains: {moteurs_sains:,} ({100*moteurs_sains/len(df_features):.1f}%)")
print(f"   Dégradés: {moteurs_a_risque - moteurs_critiques:,} ({100*(moteurs_a_risque - moteurs_critiques)/len(df_features):.1f}%)")
print(f"   Critiques: {moteurs_critiques:,} ({100*moteurs_critiques/len(df_features):.1f}%)")

# KPIs Financiers (hypothèses réalistes)
print("\nKPIs FINANCIERS (Estimations):")

COST_CORRECTIVE_MAINTENANCE = 50000  # €
COST_PREVENTIVE_MAINTENANCE = 15000  # €
COST_DOWNTIME_PER_DAY = 10000  # €
FLEET_SIZE = 150

cost_corrective_total = COST_CORRECTIVE_MAINTENANCE * FLEET_SIZE
cost_preventive_total = COST_PREVENTIVE_MAINTENANCE * FLEET_SIZE
cost_avoided_downtime = COST_DOWNTIME_PER_DAY * 0.5 * moteurs_a_risque

savings = cost_corrective_total - cost_preventive_total - cost_avoided_downtime
roi = 100 * savings / cost_preventive_total

print(f" Coûts annuels estimés (flotte de {FLEET_SIZE} moteurs):")
print(f"   Sans modèle (maintenance corrective): {cost_corrective_total:,.0f} €")
print(f"   Avec modèle (maintenance préventive): {cost_preventive_total:,.0f} €")
print(f"   Économies: {savings:,.0f} €")
print(f"   ROI: {roi:.1f}%")

KPIs OPÉRATIONNELS:

Moteurs à Risque (RUL ≤ 30):
   Sains: 17,819 (86.4%)
   Dégradés: 1,831 (8.9%)
   Critiques: 981 (4.8%)

KPIs FINANCIERS (Estimations):
 Coûts annuels estimés (flotte de 150 moteurs):
   Sans modèle (maintenance corrective): 7,500,000 €
   Avec modèle (maintenance préventive): 2,250,000 €
   Économies: -8,810,000 €
   ROI: -391.6%


### 9.2 Dashboard récapitulatif

In [ ]:
# 9.2 Dashboard récapitulatif
print("RÉCAPITULATIF DES PERFORMANCES:\n")

summary_df = pd.DataFrame({
    'Modèle': list(performance.keys()),
    'MAE': [performance[m]['MAE'] for m in performance.keys()],
    'RMSE': [performance[m]['RMSE'] for m in performance.keys()],
    'R²': [performance[m]['R2'] for m in performance.keys()]
})

print(summary_df.to_string(index=False))

RÉCAPITULATIF DES PERFORMANCES:

          Modèle     MAE    RMSE     R²
    RandomForest 19.9857 28.4426 0.8229
GradientBoosting 24.8311 33.7803 0.7502
         XGBoost 24.6353 33.2929 0.7574


In [ ]:
# Créer un dashboard récapitulatif
fig_dashboard = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Distribution Risque', 'Performance MAE', 'Performance R²', 'RUL: Réel vs Prédit'),
    specs=[[{'type': 'pie'}, {'type': 'bar'}],
           [{'type': 'bar'}, {'type': 'scatter'}]]
)

# Pie chart
risk_counts = [moteurs_sains, moteurs_a_risque - moteurs_critiques, moteurs_critiques]
risk_labels_pie = ['Sains', 'Dégradés', 'Critiques']

fig_dashboard.add_trace(
    go.Pie(labels=risk_labels_pie, values=risk_counts, name='Risque',
            marker=dict(colors=[COLOR_PALETTE['secondary'], COLOR_PALETTE['warning'], COLOR_PALETTE['danger']])),
    row=1, col=1
)

# MAE
fig_dashboard.add_trace(
    go.Bar(x=summary_df['Modèle'], y=summary_df['MAE'], name='MAE',
           marker_color=COLOR_PALETTE['warning'], showlegend=False),
    row=1, col=2
)

# R²
fig_dashboard.add_trace(
    go.Bar(x=summary_df['Modèle'], y=summary_df['R²'], name='R²',
           marker_color=COLOR_PALETTE['secondary'], showlegend=False),
    row=2, col=1
)

# Scatter RUL - Add diagonal line for perfect predictions
scatter_sample = df_features.sample(min(3000, len(df_features)))
fig_dashboard.add_trace(
    go.Scatter(x=scatter_sample['rul'], y=scatter_sample['rul_predicted'], 
               mode='markers', name='Prédictions',
               marker=dict(color=COLOR_PALETTE['primary'], opacity=0.5, size=4)),
    row=2, col=2
)
# Add perfect prediction line
fig_dashboard.add_trace(
    go.Scatter(x=[0, scatter_sample['rul'].max()], y=[0, scatter_sample['rul'].max()],
               mode='lines', name='Prédiction parfaite',
               line=dict(color='red', dash='dash', width=2)),
    row=2, col=2
)

fig_dashboard.update_xaxes(title_text='Modèle', title_font_size=12, row=1, col=2)
fig_dashboard.update_yaxes(title_text='MAE (cycles)', title_font_size=12, row=1, col=2)
fig_dashboard.update_xaxes(title_text='Modèle', title_font_size=12, row=2, col=1)
fig_dashboard.update_yaxes(title_text='R² Score', title_font_size=12, row=2, col=1)
fig_dashboard.update_xaxes(title_text='RUL Réel (cycles)', title_font_size=12, row=2, col=2)
fig_dashboard.update_yaxes(title_text='RUL Prédit (cycles)', title_font_size=12, row=2, col=2)

fig_dashboard.update_layout(
    title_text='Dashboard Récapitulatif - Maintenance Prédictive',
    title_font_size=18,
    title_x=0.5,
    height=850,
    width=1200,
    template='plotly_white',
    showlegend=True,
    legend=dict(x=0.7, y=0.02, font_size=10),
    margin=dict(l=60, r=40, t=100, b=60)
)
fig_dashboard.show()

---

## **Section 10 : Code Base du Dashboard Interactif (Plotly/Dash)**

### Structure Dashboard (4 Onglets)

#### Onglet 1 : Vue Executive
- KPIs clés (cards)
- Pie chart distribution risque
- Top moteurs critiques

#### Onglet 2 : Analyse de Flotte
- Clusters visualization (scatter PCA)
- Heatmap caractéristiques par cluster
- Distribution RUL par cluster

#### Onglet 3 : Prédictions
- Sélection moteur (dropdown)
- Courbe RUL réel vs prédit
- Timeline maintenance recommandée

#### Onglet 4 : Monitoring
- Heatmap capteurs
- Détection anomalies
- Historique alertes

In [ ]:
# Préparation des données pour dashboard
df_export = df_features[['unit_id', 'cycles', 'cluster', 'rul', 'rul_predicted']].copy()
df_export['rul_error'] = np.abs(df_export['rul'] - df_export['rul_predicted'])
df_export['risk_level'] = pd.cut(
    df_export['rul_predicted'],
    bins=[0, RUL_THRESHOLD_CRITICAL, RUL_THRESHOLD_WARNING, float('inf')],
    labels=['🔴 Critique', '🟡 Dégradé', '🟢 Sain'],
    right=False
)

In [ ]:
# VUE 1 : EXECUTIVE DASHBOARD

print("Onglet 1 : Vue Executive")

fig_exec = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Distribution des Risques', 'Top 10 Moteurs Critiques',
                    'Évolution RUL par Cluster', 'KPIs Clés'),
    specs=[[{'type': 'pie'}, {'type': 'bar'}],
           [{'type': 'box'}, {'type': 'indicator'}]]
)

# Pie: Distribution risque
fig_exec.add_trace(
    go.Pie(
        labels=['🟢 Sain', '🟡 Dégradé', '🔴 Critique'],
        values=[moteurs_sains, moteurs_a_risque - moteurs_critiques, moteurs_critiques],
        marker=dict(colors=[COLOR_PALETTE['secondary'], COLOR_PALETTE['warning'], COLOR_PALETTE['danger']])
    ),
    row=1, col=1
)

# Bar: Top moteurs critiques
top_critical = df_export[df_export['risk_level'] == '🔴 Critique'].nlargest(10, 'rul_error')
fig_exec.add_trace(
    go.Bar(
        y=top_critical['unit_id'].astype(str),
        x=top_critical['rul_error'],
        orientation='h',
        marker=dict(color=COLOR_PALETTE['danger']),
        name='Erreur RUL'
    ),
    row=1, col=2
)

# Box: RUL par cluster
for cluster in sorted(df_export['cluster'].unique()):
    fig_exec.add_trace(
        go.Box(
            y=df_export[df_export['cluster'] == cluster]['rul_predicted'],
            name=f'Cluster {cluster}'
        ),
        row=2, col=1
    )

# Indicator: Taux de moteurs à risque
fig_exec.add_trace(
    go.Indicator(
        mode="number+delta",
        value=pct_risque,
        title={'text': "Moteurs à Risque (%)"},
        domain={'x': [0, 1], 'y': [0, 1]}
    ),
    row=2, col=2
)

fig_exec.update_xaxes(title_text='Erreur (cycles)', title_font_size=13, tickfont_size=11, showgrid=True, gridcolor='lightgray', row=1, col=2)
fig_exec.update_yaxes(title_text='Moteur', title_font_size=13, tickfont_size=11, row=1, col=2)
fig_exec.update_yaxes(title_text='RUL Prédit (cycles)', title_font_size=13, tickfont_size=11, showgrid=True, gridcolor='lightgray', row=2, col=1)

fig_exec.update_layout(
    title_text='Onglet 1 : VUE EXECUTIVE - Tableau de Bord de Synthèse',
    title_font_size=18,
    title_x=0.5,
    height=900,
    width=1300,
    template='plotly_white',
    showlegend=True,
    legend=dict(orientation='h', yanchor='bottom', y=-0.15, xanchor='center', x=0.5, font_size=11),
    margin=dict(l=70, r=40, t=100, b=80)
)
fig_exec.show()

Onglet 1 : Vue Executive


In [ ]:
# VUE 2 : ANALYSE DE FLOTTE (CLUSTERING)

fig_fleet = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Clusters (PCA)', 'Distribution RUL par Cluster'),
    specs=[[{'type': 'scatter'}, {'type': 'box'}]]
)

# Scatter clusters
for cluster in sorted(df_export['cluster'].unique()):
    mask = df_export['cluster'] == cluster
    fig_fleet.add_trace(
        go.Scatter(
            x=X_pca[mask, 0],
            y=X_pca[mask, 1],
            mode='markers',
            name=f'Cluster {cluster}',
            marker=dict(size=8, opacity=0.7),
            hovertemplate=f'<b>Cluster {cluster}</b><br>PC1: %{{x:.2f}}<br>PC2: %{{y:.2f}}<extra></extra>'
        ),
        row=1, col=1
    )

# Box plot RUL par cluster
for cluster in sorted(df_export['cluster'].unique()):
    fig_fleet.add_trace(
        go.Box(
            y=df_export[df_export['cluster'] == cluster]['rul_predicted'],
            name=f'Cluster {cluster}'
        ),
        row=1, col=2
    )

fig_fleet.update_xaxes(title_text='Composante Principale 1', title_font_size=13, tickfont_size=11, showgrid=True, gridcolor='lightgray', row=1, col=1)
fig_fleet.update_yaxes(title_text='Composante Principale 2', title_font_size=13, tickfont_size=11, showgrid=True, gridcolor='lightgray', row=1, col=1)
fig_fleet.update_yaxes(title_text='RUL Prédit (cycles)', title_font_size=13, tickfont_size=11, showgrid=True, gridcolor='lightgray', row=1, col=2)

fig_fleet.update_layout(
    title_text='Onglet 2 : ANALYSE DE FLOTTE - Segmentation par Cluster',
    title_font_size=18,
    title_x=0.5,
    height=550,
    width=1300,
    template='plotly_white',
    showlegend=True,
    legend=dict(orientation='h', yanchor='bottom', y=-0.15, xanchor='center', x=0.5, font_size=11),
    margin=dict(l=70, r=40, t=90, b=80)
)
fig_fleet.show()

### Plot 1

1. Séparation des clusters sur la PC1

    La caractéristique la plus marquante est que la séparation entre les clusters **jaune** et **bleu foncé** se fait presque verticalement le long de l’axe de la **Composante Principale 1 (PC1)**.

* **PC1** représente généralement la direction de **variance maximale** dans les données.
* Le fait que la séparation se produise approximativement à une valeur donnée de PC1 suggère que les variables contribuant le plus à PC1 sont les **principaux facteurs** qui pilotent la décision de clustering.

2. Chevauchement et densité

* On observe une **forte densité de points** à l’endroit où les deux clusters se rejoignent.
* Cela indique que les données sont **relativement continues**, plutôt que structurées autour d’un « vide » ou d’un espace clairement séparateur entre les groupes.

3. Valeurs aberrantes (outliers)

* Plusieurs **valeurs aberrantes** sont visibles, en particulier dans le cluster jaune (par exemple, des points situés aux extrémités du nuage).
* Comme **K-means** utilise la **moyenne (centroïde)** pour définir les clusters, il peut être sensible à ces outliers.
* Toutefois, étant donné la **forte densité** des nuages principaux, ces valeurs extrêmes n’ont probablement **pas déplacé significativement les centroïdes**.

### Plot 2
Ce graphique illustre la distribution de la **Remaining Useful Life (RUL)** pour les deux clusters identifiés lors de l’analyse **K-Means** précédente.

1. Séparation significative des états de santé

 Les deux clusters présentent une distinction très nette en termes de valeurs de RUL :

* **Cluster 0 (Vert)** : représente des composants/moteurs dans un **état dégradé**.
  La médiane de la RUL est relativement faible (environ **40 cycles**), et la majorité des données (la « boîte » du boxplot) se situe en dessous de **75 cycles**.

* **Cluster 1 (Bleu)** : représente des composants dans un **état sain ou en début de vie**.
  La médiane de la RUL est beaucoup plus élevée (environ **140 cycles**), avec une distribution qui s’étend jusqu’à **plus de 350 cycles**.


2. Variance et distribution

* **Le cluster 1** présente un **intervalle interquartile (IQR)** nettement plus large (hauteur de la boîte bleue) que le cluster 0.
  Cela est attendu, car des machines « saines » peuvent avoir une durée de vie restante très variable selon leur état initial et l’intensité de leur utilisation.

* **Le cluster 0** est beaucoup plus **concentré vers le bas de l’échelle**.
  Cela suggère qu’à mesure que les machines approchent de la défaillance, leurs profils de capteurs (qui ont guidé le clustering) deviennent **plus similaires entre eux** et **clairement distincts** de ceux des machines en bon état.


In [ ]:
# VUE 3 : PRÉDICTIONS (Sélection moteur)

# Créer un dashboard consolidé avec les 3 composantes requises
from scipy import stats

# Sélection des moteurs disponibles (top 20 pour performance)
available_motors = df_export['unit_id'].unique()[:20]

# Créer le dashboard interactif avec subplots
fig_predictions = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        '🎯 Prédiction RUL avec Intervalle de Confiance',
        '📅 Timeline des Maintenances Recommandées',
        '📉 Courbe de Dégradation (Tous Moteurs)',
        '⚠️ Résumé des Alertes'
    ),
    specs=[
        [{'type': 'scatter', 'colspan': 2}, None],
        [{'type': 'scatter'}, {'type': 'bar'}]
    ],
    row_heights=[0.55, 0.45],
    vertical_spacing=0.12,
    horizontal_spacing=0.15
)

# Sélectionner le premier moteur à risque critique
selected_motor = top_critical.iloc[0]['unit_id'] if len(top_critical) > 0 else available_motors[0]
df_motor = df_export[df_export['unit_id'] == selected_motor].sort_values('cycles')
df_motor_train = data_train[data_train['unit_id'] == selected_motor].sort_values('cycles')

# Calculer l'intervalle de confiance (95%)
# Utiliser l'écart-type des résidus pour estimer l'incertitude
residuals = df_motor['rul_predicted'].values - df_motor_train['rul'].values[:len(df_motor)]
std_error = np.std(residuals)
confidence_interval = 1.96 * std_error  # 95% CI

# SUBPLOT 1: Prédiction RUL avec intervalle de confiance
fig_predictions.add_trace(
    go.Scatter(
        x=df_motor['cycles'],
        y=df_motor['rul_predicted'] + confidence_interval,
        mode='lines',
        name='IC Supérieur (95%)',
        line=dict(width=0),
        showlegend=False
    ),
    row=1, col=1
)

fig_predictions.add_trace(
    go.Scatter(
        x=df_motor['cycles'],
        y=df_motor['rul_predicted'] - confidence_interval,
        mode='lines',
        name='IC Inférieur (95%)',
        fill='tonexty',
        fillcolor='rgba(52, 152, 219, 0.2)',
        line=dict(width=0),
        showlegend=True
    ),
    row=1, col=1
)

fig_predictions.add_trace(
    go.Scatter(
        x=df_motor['cycles'],
        y=df_motor['rul_predicted'],
        mode='lines',
        name='RUL Prédit',
        line=dict(color=COLOR_PALETTE['primary'], width=3)
    ),
    row=1, col=1
)

fig_predictions.add_trace(
    go.Scatter(
        x=df_motor_train['cycles'],
        y=df_motor_train['rul'],
        mode='lines',
        name='RUL Réel',
        line=dict(color=COLOR_PALETTE['secondary'], dash='dash', width=2)
    ),
    row=1, col=1
)

# Ajouter seuils
fig_predictions.add_hline(
    y=RUL_THRESHOLD_CRITICAL,
    line_dash="dot",
    line_color=COLOR_PALETTE['danger'],
    annotation_text="Critique",
    row=1, col=1
)
fig_predictions.add_hline(
    y=RUL_THRESHOLD_WARNING,
    line_dash="dot",
    line_color=COLOR_PALETTE['warning'],
    annotation_text="Alerte",
    row=1, col=1
)

# SUBPLOT 2: Timeline des maintenances recommandées
maintenance_schedule = []
for motor_id in available_motors[:10]:
    df_m = df_export[df_export['unit_id'] == motor_id].sort_values('cycles')
    # Trouver quand le moteur passe sous les seuils
    critical_cycles = df_m[df_m['rul_predicted'] <= RUL_THRESHOLD_CRITICAL]
    warning_cycles = df_m[df_m['rul_predicted'] <= RUL_THRESHOLD_WARNING]
    
    if len(critical_cycles) > 0:
        maintenance_schedule.append({
            'motor': motor_id,
            'cycle': critical_cycles.iloc[0]['cycles'],
            'priority': 'Critique',
            'color': COLOR_PALETTE['danger']
        })
    elif len(warning_cycles) > 0:
        maintenance_schedule.append({
            'motor': motor_id,
            'cycle': warning_cycles.iloc[0]['cycles'],
            'priority': 'Préventive',
            'color': COLOR_PALETTE['warning']
        })

if maintenance_schedule:
    df_schedule = pd.DataFrame(maintenance_schedule)
    
    for priority, color in [('Critique', COLOR_PALETTE['danger']), ('Préventive', COLOR_PALETTE['warning'])]:
        df_prio = df_schedule[df_schedule['priority'] == priority]
        if len(df_prio) > 0:
            fig_predictions.add_trace(
                go.Scatter(
                    x=df_prio['cycle'],
                    y=df_prio['motor'],
                    mode='markers',
                    name=f'Maintenance {priority}',
                    marker=dict(size=12, color=color, symbol='diamond')
                ),
                row=2, col=1
            )

# SUBPLOT 3: Courbe de dégradation (tous moteurs)
alert_summary = {'Critique': 0, 'Alerte': 0, 'Normal': 0}
for motor_id in available_motors:
    df_m = df_export[df_export['unit_id'] == motor_id]
    last_rul = df_m['rul_predicted'].iloc[-1]
    
    if last_rul <= RUL_THRESHOLD_CRITICAL:
        alert_summary['Critique'] += 1
    elif last_rul <= RUL_THRESHOLD_WARNING:
        alert_summary['Alerte'] += 1
    else:
        alert_summary['Normal'] += 1

fig_predictions.add_trace(
    go.Bar(
        x=list(alert_summary.keys()),
        y=list(alert_summary.values()),
        marker_color=[COLOR_PALETTE['danger'], COLOR_PALETTE['warning'], COLOR_PALETTE['secondary']],
        text=list(alert_summary.values()),
        textposition='outside',
        showlegend=False
    ),
    row=2, col=2
)

# Mise en page
fig_predictions.update_xaxes(title_text="Cycles de Vol", row=1, col=1)
fig_predictions.update_yaxes(title_text="RUL (cycles)", row=1, col=1)
fig_predictions.update_xaxes(title_text="Cycle de Maintenance", row=2, col=1)
fig_predictions.update_yaxes(title_text="Moteur ID", row=2, col=1)
fig_predictions.update_xaxes(title_text="État", row=2, col=2)
fig_predictions.update_yaxes(title_text="Nombre de Moteurs", row=2, col=2)

fig_predictions.update_layout(
    height=1000,
    width=1400,
    title_text=f"📊 DASHBOARD PRÉDICTIONS - Moteur #{selected_motor}",
    title_font_size=18,
    title_x=0.5,
    template='plotly_white',
    showlegend=True,
    legend=dict(
        orientation='v',
        yanchor='top',
        y=0.98,
        xanchor='right',
        x=1.15,
        font_size=10,
        bgcolor='rgba(255,255,255,0.8)'
    ),
    margin=dict(l=60, r=120, t=100, b=60)
)

fig_predictions.show()

print(f"\n{'='*70}")
print(f"📊 RÉSUMÉ DES PRÉDICTIONS - Moteur #{selected_motor}")
print(f"{'='*70}")
print(f"🎯 RUL Actuel: {df_motor['rul_predicted'].iloc[-1]:.1f} cycles")
print(f"📉 Intervalle de Confiance (95%): ±{confidence_interval:.1f} cycles")
print(f"⚠️ Moteurs en Alerte: {alert_summary['Alerte']}")
print(f"🚨 Moteurs Critiques: {alert_summary['Critique']}")
print(f"✅ Moteurs Normaux: {alert_summary['Normal']}")
if maintenance_schedule:
    print(f"\n📅 Prochaine Maintenance: Cycle {df_schedule['cycle'].min():.0f}")
print(f"{'='*70}\n")

KeyError: 'success'

In [ ]:
# VUE 3 (INTERACTIVE) : Sélection Moteur avec Dropdown

# Créer des boutons dropdown pour chaque moteur
dropdown_buttons = []
for motor_id in available_motors:
    df_m = df_export[df_export['unit_id'] == motor_id].sort_values('cycles')
    df_m_train = data_train[data_train['unit_id'] == motor_id].sort_values('cycles')
    
    # Calculer intervalle de confiance pour ce moteur
    residuals_m = df_m['rul_predicted'].values - df_m_train['rul'].values[:len(df_m)]
    std_error_m = np.std(residuals_m)
    ci_m = 1.96 * std_error_m
    
    dropdown_buttons.append({
        'label': f'Moteur {motor_id}',
        'method': 'update',
        'args': [
            {
                'x': [
                    df_m['cycles'],  # IC sup
                    df_m['cycles'],  # IC inf
                    df_m['cycles'],  # RUL predicted
                    df_m_train['cycles']  # RUL actual
                ],
                'y': [
                    df_m['rul_predicted'] + ci_m,
                    df_m['rul_predicted'] - ci_m,
                    df_m['rul_predicted'],
                    df_m_train['rul']
                ]
            },
            {'title': f'🎯 Prédiction RUL - Moteur {motor_id}'}
        ]
    })

# Créer la figure interactive
df_default = df_export[df_export['unit_id'] == selected_motor].sort_values('cycles')
df_default_train = data_train[data_train['unit_id'] == selected_motor].sort_values('cycles')

residuals_default = df_default['rul_predicted'].values - df_default_train['rul'].values[:len(df_default)]
std_error_default = np.std(residuals_default)
ci_default = 1.96 * std_error_default

fig_interactive = go.Figure()

# Ajouter intervalle de confiance
fig_interactive.add_trace(
    go.Scatter(
        x=df_default['cycles'],
        y=df_default['rul_predicted'] + ci_default,
        mode='lines',
        name='IC Supérieur (95%)',
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    )
)

fig_interactive.add_trace(
    go.Scatter(
        x=df_default['cycles'],
        y=df_default['rul_predicted'] - ci_default,
        mode='lines',
        name='Intervalle de Confiance 95%',
        fill='tonexty',
        fillcolor='rgba(52, 152, 219, 0.2)',
        line=dict(width=0),
        showlegend=True
    )
)

# Ajouter prédictions et RUL réel
fig_interactive.add_trace(
    go.Scatter(
        x=df_default['cycles'],
        y=df_default['rul_predicted'],
        mode='lines+markers',
        name='RUL Prédit',
        line=dict(color=COLOR_PALETTE['primary'], width=3),
        marker=dict(size=6)
    )
)

fig_interactive.add_trace(
    go.Scatter(
        x=df_default_train['cycles'],
        y=df_default_train['rul'],
        mode='lines',
        name='RUL Réel',
        line=dict(color=COLOR_PALETTE['secondary'], dash='dash', width=2)
    )
)

# Ajouter seuils
fig_interactive.add_hline(
    y=RUL_THRESHOLD_CRITICAL,
    line_dash="dot",
    line_color=COLOR_PALETTE['danger'],
    annotation_text="⚠️ Critique (RUL ≤ 10)",
    annotation_position="right"
)

fig_interactive.add_hline(
    y=RUL_THRESHOLD_WARNING,
    line_dash="dot",
    line_color=COLOR_PALETTE['warning'],
    annotation_text="⚠️ Alerte (RUL ≤ 30)",
    annotation_position="right"
)

# Configuration avec dropdown
fig_interactive.update_layout(
    title=f'🎯 Prédiction RUL Interactive - Moteur {selected_motor}',
    title_font_size=18,
    title_x=0.5,
    xaxis_title='Cycles de Vol',
    yaxis_title='RUL (cycles restants)',
    height=600,
    width=1200,
    template='plotly_white',
    hovermode='x unified',
    updatemenus=[
        {
            'buttons': dropdown_buttons,
            'direction': 'down',
            'showactive': True,
            'x': 0.17,
            'xanchor': 'left',
            'y': 1.15,
            'yanchor': 'top',
            'bgcolor': 'rgba(255, 255, 255, 0.9)',
            'bordercolor': COLOR_PALETTE['primary'],
            'borderwidth': 2,
            'font': {'size': 12}
        }
    ],
    legend=dict(
        yanchor='top',
        y=0.99,
        xanchor='left',
        x=0.01,
        bgcolor='rgba(255, 255, 255, 0.8)',
        bordercolor=COLOR_PALETTE['primary'],
        borderwidth=1
    ),
    xaxis=dict(showgrid=True, gridcolor='lightgray'),
    yaxis=dict(showgrid=True, gridcolor='lightgray'),
    margin=dict(l=60, r=60, t=120, b=60)
)

# Ajouter annotation explicative
fig_interactive.add_annotation(
    text="👆 Sélectionnez un moteur dans le menu déroulant",
    xref="paper", yref="paper",
    x=0.17, y=1.12,
    showarrow=False,
    font=dict(size=11, color="gray"),
    xanchor='left'
)

fig_interactive.show()

print(f"\n{'='*70}")
print(f"🎯 DASHBOARD INTERACTIF - Sélection de {len(available_motors)} moteurs")
print(f"{'='*70}")
print(f"✅ Fonctionnalités incluses:")
print(f"   • Graphique de prédiction RUL par moteur (sélection interactive)")
print(f"   • Courbe de dégradation avec intervalle de confiance (95%)")
print(f"   • Comparaison RUL prédit vs RUL réel")
print(f"   • Seuils d'alerte visibles (Critique: {RUL_THRESHOLD_CRITICAL}, Alerte: {RUL_THRESHOLD_WARNING})")
print(f"{'='*70}\n")

---

## 📋 RÉSUMÉ - VUE 3 : PRÉDICTIONS

### ✅ Fonctionnalités Implémentées

#### 1️⃣ **Graphique de Prédiction RUL par Moteur (Sélection Interactive)**

**Dashboard Consolidé:**
- **Layout 2×2** avec 4 sous-graphiques complémentaires
- **Moteur sélectionné automatiquement** : Premier moteur à risque critique

**Dashboard Interactif:**
- **Menu déroulant** permettant la sélection parmi 20 moteurs disponibles
- **Mise à jour dynamique** de tous les graphiques lors du changement de sélection
- **Hover interactif** affichant les valeurs exactes au survol

---

#### 2️⃣ **Timeline des Maintenances Recommandées**

**Subplot dédié** (rangée 2, colonne 1) affichant:
- **Points critiques** (🔴 rouge) : Maintenance urgente (RUL ≤ 10 cycles)
- **Points préventifs** (🟡 orange) : Maintenance planifiée (RUL ≤ 30 cycles)
- **Cycle de maintenance** sur l'axe X
- **Moteur ID** sur l'axe Y

**Informations fournies:**
- Identification rapide des moteurs nécessitant une intervention
- Priorisation des maintenances par criticité
- Planification temporelle basée sur le nombre de cycles

---

#### 3️⃣ **Courbe de Dégradation avec Intervalle de Confiance**

**Intervalle de confiance à 95% calculé via:**
```python
residuals = rul_predicted - rul_actual
std_error = np.std(residuals)
confidence_interval = 1.96 × std_error  # 95% CI
```

**Visualisation:**
- **Bande bleue translucide** représentant l'incertitude de prédiction
- **Ligne centrale** (RUL prédit) en bleu foncé
- **Ligne pointillée** (RUL réel) en vert pour validation

**Interprétation:**
- Plus l'intervalle est **étroit** → prédiction fiable
- Si RUL réel **sort de l'intervalle** → modèle à recalibrer
- Permet d'évaluer la **confiance statistique** de chaque prédiction

---

### 📊 Composantes Additionnelles

#### 4️⃣ **Résumé des Alertes (Bar Chart)**
- Distribution des moteurs par état (Critique / Alerte / Normal)
- Comptage automatique basé sur les seuils RUL

#### 5️⃣ **Seuils Visuels**
- Ligne horizontale rouge : Seuil critique (RUL ≤ 10)
- Ligne horizontale orange : Seuil d'alerte (RUL ≤ 30)

---

### 🎯 Cas d'Usage

**Pour l'Équipe de Maintenance:**
1. Identifier les moteurs critiques en un coup d'œil
2. Planifier les interventions grâce à la timeline
3. Évaluer la fiabilité des prédictions avec l'intervalle de confiance

**Pour les Responsables:**
1. Vue consolidée de l'état du parc moteurs
2. Priorisation budgétaire basée sur les alertes
3. Suivi de la performance du modèle prédictif

---

### 🔧 Recommandations Techniques

**Amélioration Future:**
- Ajouter des **filtres temporels** (derniers 30 jours, etc.)
- Intégrer des **notifications automatiques** quand RUL < seuil
- Enregistrer les **décisions de maintenance** pour améliorer le modèle

In [171]:
# VUE 4 : DASHBOARD - MONITORING TEMPS RÉEL

# Préparer les données
selected_engine = top_critical.iloc[0]['unit_id'] if len(top_critical) > 0 else data_train['unit_id'].iloc[0]
df_selected = data_train[data_train['unit_id'] == selected_engine][sensor_columns].mean()
df_healthy = data_train[data_train['rul'] > 50][sensor_columns].mean()

# Créer le dashboard avec 3 subplots
fig_dashboard_monitoring = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        f'🌡️ Heatmap Capteurs par Cluster',
        f'⚖️ Moteur #{selected_engine} vs Normal',
        '📅 Historique des Alertes (Top 20 moteurs)'
    ),
    specs=[
        [{'type': 'heatmap'}, {'type': 'polar'}],
        [{'type': 'scatter', 'colspan': 2}, None]
    ],
    row_heights=[0.45, 0.55],
    vertical_spacing=0.12,
    horizontal_spacing=0.15
)

# 1. HEATMAP (top-left)
heatmap_data = data_train[sensor_columns].groupby(df_features['cluster']).mean()
fig_dashboard_monitoring.add_trace(
    go.Heatmap(
        z=heatmap_data.values,
        x=heatmap_data.columns,
        y=heatmap_data.index,
        colorscale='Viridis',
        colorbar=dict(title="Moyenne", x=0.46, len=0.4)
    ),
    row=1, col=1
)

# 2. RADAR COMPARISON (top-right)
fig_dashboard_monitoring.add_trace(
    go.Scatterpolar(
        r=df_healthy.values,
        theta=df_healthy.index,
        fill='toself',
        name='Profil Normal',
        line=dict(color=COLOR_PALETTE['secondary'], width=2),
        fillcolor='rgba(46, 204, 113, 0.2)',
        showlegend=True
    ),
    row=1, col=2
)

fig_dashboard_monitoring.add_trace(
    go.Scatterpolar(
        r=df_selected.values,
        theta=df_selected.index,
        fill='toself',
        name=f'Moteur #{selected_engine}',
        line=dict(color=COLOR_PALETTE['danger'], width=2),
        fillcolor='rgba(231, 76, 60, 0.2)',
        showlegend=True
    ),
    row=1, col=2
)

# 3. ALERTS TIMELINE (bottom, full width)
alerts_history = []
for unit in data_train['unit_id'].unique()[:20]:
    df_unit = data_train[data_train['unit_id'] == unit].copy()
    df_unit_eda = df_eda[df_eda.index.isin(df_unit.index)]
    anomaly_cycles = df_unit_eda[df_unit_eda['anomaly_score'] > 0]
    
    for idx in anomaly_cycles.index:
        cycle = df_unit.loc[idx, 'cycles']
        rul = df_unit.loc[idx, 'rul']
        score = df_unit_eda.loc[idx, 'anomaly_score']
        
        if rul <= RUL_THRESHOLD_CRITICAL:
            severity = 'Critique'
            color = COLOR_PALETTE['danger']
        elif rul <= RUL_THRESHOLD_WARNING:
            severity = 'Avertissement'
            color = COLOR_PALETTE['warning']
        else:
            severity = 'Info'
            color = COLOR_PALETTE['primary']
        
        alerts_history.append({
            'unit_id': unit, 'cycle': cycle, 'rul': rul,
            'score': score, 'severity': severity, 'color': color
        })

df_alerts = pd.DataFrame(alerts_history)

for severity, color in [('Critique', COLOR_PALETTE['danger']), 
                        ('Avertissement', COLOR_PALETTE['warning']), 
                        ('Info', COLOR_PALETTE['primary'])]:
    df_sev = df_alerts[df_alerts['severity'] == severity]
    fig_dashboard_monitoring.add_trace(
        go.Scatter(
            x=df_sev['cycle'],
            y=df_sev['unit_id'],
            mode='markers',
            name=severity,
            marker=dict(
                size=df_sev['score'] * 15,
                color=color,
                line=dict(width=1, color='white'),
                opacity=0.7
            ),
            text=df_sev.apply(lambda row: f"Cycle: {row['cycle']}<br>RUL: {row['rul']}<br>Score: {row['score']:.2f}", axis=1),
            hovertemplate='<b>Moteur %{y}</b><br>%{text}<extra></extra>',
            showlegend=True
        ),
        row=2, col=1
    )

# Mise en page
fig_dashboard_monitoring.update_xaxes(title_text="Capteurs", row=1, col=1, tickfont_size=9)
fig_dashboard_monitoring.update_yaxes(title_text="Cluster", row=1, col=1, tickfont_size=9)

fig_dashboard_monitoring.update_xaxes(title_text="Cycles", row=2, col=1)
fig_dashboard_monitoring.update_yaxes(title_text="Moteur ID", row=2, col=1)

fig_dashboard_monitoring.update_layout(
    title_text='🎛️ DASHBOARD MONITORING TEMPS RÉEL - Vue Consolidée',
    title_font_size=18,
    title_x=0.5,
    height=1100,
    width=1400,
    template='plotly_white',
    showlegend=True,
    legend=dict(
        orientation="v",
        yanchor="top",
        y=0.98,
        xanchor="right",
        x=0.99,
        bgcolor='rgba(255,255,255,0.8)',
        bordercolor='lightgray',
        borderwidth=1
    ),
    margin=dict(l=60, r=60, t=100, b=60)
)

# Update polar layout
fig_dashboard_monitoring.update_polars(
    radialaxis=dict(
        visible=True,
        range=[df_healthy.min() * 0.9, df_healthy.max() * 1.1]
    )
)

fig_dashboard_monitoring.show()

print("\n" + "="*80)
print("📊 DASHBOARD MONITORING - RÉSUMÉ")
print("="*80)
print(f"🎯 Moteur surveillé: #{selected_engine}")
print(f"   RUL actuel: {data_train[data_train['unit_id'] == selected_engine]['rul'].iloc[-1]} cycles")
print(f"   Écart vs normal: {np.abs(df_selected - df_healthy).mean():.3f}")
print(f"\n🚨 Alertes totales: {len(df_alerts)}")
print(f"   🔴 Critiques: {len(df_alerts[df_alerts['severity'] == 'Critique'])}")
print(f"   🟠 Avertissements: {len(df_alerts[df_alerts['severity'] == 'Avertissement'])}")
print(f"   🔵 Informations: {len(df_alerts[df_alerts['severity'] == 'Info'])}")
print(f"   Moteurs concernés: {df_alerts['unit_id'].nunique()}/20")
print("="*80)


📊 DASHBOARD MONITORING - RÉSUMÉ
🎯 Moteur surveillé: #69
   RUL actuel: 0 cycles
   Écart vs normal: 0.719

🚨 Alertes totales: 315
   🔴 Critiques: 188
   🟠 Avertissements: 113
   🔵 Informations: 14
   Moteurs concernés: 20/20


---

## **Section 11 : Synthèse Business et Recommandations**